<a href="https://colab.research.google.com/github/invisilico/ActivityExtractor/blob/master/Rapid_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the RAPID Android Activity TimeStamps Extractor!

Inspired by the De La Iglesias Lab project on E-mail Timestamps, this notebook helps you easily extract, clean and visualise your Android phone's activity data based on timestamps on when apps were opened. 

A pandas dataframe is created with the following structure and is saved as CSV at the end of the notebook.


---


```
# dataframe structure

   App     Year    Month  Date  Time  of24h
0  App     2020    7      10    1534  15.566667
1  App     2020    7      10    1434  14.566667
2  App     2020    7      11    1534  15.566667
```


---


Instructions for using colab: the [ ] on the left of each "block"/"cell" runs them. Run them in order and wait for them to finish. If you run into errors, feel free to contact me at nishantjana5@gmail.com 


---


Made by Nishant Jana during SRBR ChronoSchool 2020.

Twitter: @in_visilico, Github: @invisilico


---



In [2]:
#@title one cell to do it ALL
#@markdown Fill in the inputs below before running the cell.
#@markdown
#@markdown When the "choose file" button appears, click it and choose your 'My Activity.json' file to upload. Please wait for it to finish, sip your coffee while the notebook does all your work for you!
#set-up
import json
import csv
import pandas as pd
import numpy as np
import re
from google.colab import files
import matplotlib.pyplot as plt
from calendar import monthrange
from calendar import isleap

#load file
file = files.upload()
name = list(file.keys())[0] 
data = json.loads(file[name]) #loads data from JSON
print("Data Loaded from JSON File.")

!rm *.* #removes file, bash shell command
print("JSON File deleted from Colab, verify in files on left panel.")

#build dataframe with timezone adjust
timezone = 5.5 #@param {type:"number"}
hoffset = np.floor(timezone) * (timezone/abs(timezone))
minoffset = (timezone - np.floor(timezone))*60

appname = []
datetime = []

for dict in data:
    appname += [dict['header']]
    datetime += [dict['time']]
    
appname.reverse()
datetime.reverse()

yy = []
mm = []
dd = []
time = []
of24 = []

for t in datetime:
    y = int(t[0:4])
    m = int(t[5:7])
    d = int(t[8:10])
    h = int(t[11:13]) + hoffset
    min = int(t[14:16]) + minoffset
    if min > 59:
      min -= 60
      h += 1
    if h > 23:
      h -= 24
      d += 1
    if d > monthrange(y,m)[1]:
      d -= monthrange(y,m)[1]
      m += 1
    if m > 12:
      m -= 12
      y += 1
    yy += [y]
    mm += [m]
    dd += [d]
    time += [h*100 + min]
    of24 += [h + (min/60)]

dataframe = pd.DataFrame(list(zip(appname,yy,mm,dd,time,of24)),columns = ['App','Year','Month','Date','Time','of24h'])
print('dataframe created!')

#privacy Filter
dataframe.loc[dataframe['App'].str.contains('clock', case=False), 'App'] = 'clock'
dataframe.loc[dataframe['App'] != "clock", "App"] = "app"

print("appnames hae been removed, replaced with "+str(dataframe.App.unique()))

#download csv
#@markdown Set Time Zone offset in hours. ex: EDT is -4 and India is +5.5
#@markdown 
#@markdown TimeZone Adjust does not account for daylight saving. 
#@markdown 
#@markdown Type in your desired name for the output CSV file:
OutputFile = 'Data' #@param {type:"string"}
#@markdown Do not include .csv in name, just the title of your choice.
name = OutputFile+".csv"
dataframe.to_csv(name) 
files.download(name)
print("you are done!")

Saving My Activity.json to My Activity.json
Data Loaded from JSON File.
JSON File deleted from Colab, verify in files on left panel.
dataframe created!
appnames hae been removed, replaced with ['app' 'clock']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

you are done!


Once downloaded, you're all done! Head over the link below to visualize your data in all the cool ways of chronobiology!

https://circada.shinyapps.io/VisualizationDemo/